In [1]:
# xor_nn  : @tf.function 사용
# two layers of neural network

# @tf.function 장식자는 함수를 호출하면 그래프를 생성해준다

import tensorflow as tf
import numpy as np
tf.random.set_seed(5)
import time
# train data set :
x_data = [[0,0],
          [0,1],
          [1,0],
          [1,1]]

y_data = [[0],
          [1],
          [1],
          [0]]

x_train = np.array(x_data,dtype=np.float32)
y_train = np.array(y_data,dtype=np.float32)

In [2]:
# 변수 초기화 : weight, bias

#  (4,2) * (2,2) = (4,2)

start = time.time()
# Layer 1
W1 = tf.Variable(tf.random.normal([2,2]), name='weight1')
b1 = tf.Variable(tf.random.normal([2]), name='bias1')

# @tf.function
def layer1(X):
    return  tf.sigmoid(tf.matmul(X,W1) + b1 )

# layer 2
W2 = tf.Variable(tf.random.normal([2,1]), name='weight2')
b2 = tf.Variable(tf.random.normal([1]), name='bias2')

In [3]:
# hypothesis 예측 함수(방정식) 
# @tf.function
def hypothesis(X):
    return  tf.sigmoid(tf.matmul(layer1(X),W2) + b2  )
     

In [4]:
# 비용함수 : logloss
# @tf.function
def cost_func():
    cost = -tf.reduce_mean(y_train*tf.math.log(hypothesis(x_train)) + (1-y_train)*
                       tf.math.log(1-hypothesis(x_train)))
    return cost

In [5]:
# 경사 하강법
# learning rate (학습율) 을 0.01로 설정하여  optimizer객체를 생성
# optimizer = tf.compat.v1.train.GradientDescentOptimizer(learning_rate=0.01)
optimizer = tf.keras.optimizers.Adam(lr=0.01)

In [6]:
# 학습 시작
print('***** Start Learning!!')
for step in range(5001):
    # cost를 minimize한다
    optimizer.minimize(cost_func, var_list=[W1,b1,W2,b2])
    
    if step % 1000 == 0:
        print('%04d'%step, 'cost: [', cost_func().numpy(), ']  W1: ', W1.numpy(), '  b1: ', b1.numpy())
        
print('***** Learning Finished!!')

***** Start Learning!!
0000 cost: [ 0.7040971 ]  W1:  [[-0.19030488 -0.9402918 ]
 [-0.04963873 -0.73254627]]   b1:  [0.21652977 0.8206513 ]
1000 cost: [ 0.02550175 ]  W1:  [[-5.024761  -6.6669364]
 [-4.994727  -6.5032763]]   b1:  [7.5096507 2.915475 ]
2000 cost: [ 0.0068961554 ]  W1:  [[-5.944842 -7.574677]
 [-5.917539 -7.41493 ]]   b1:  [8.922688 3.408215]
3000 cost: [ 0.0029871406 ]  W1:  [[-6.4519587 -8.081456 ]
 [-6.426093  -7.9230824]]   b1:  [9.694395 3.674915]
4000 cost: [ 0.0015255288 ]  W1:  [[-6.8228345 -8.453764 ]
 [-6.797982  -8.296077 ]]   b1:  [10.256575   3.8682897]
5000 cost: [ 0.00084185496 ]  W1:  [[-7.127812  -8.760641 ]
 [-7.1037564 -8.603351 ]]   b1:  [10.717856   4.0264554]
***** Learning Finished!!


In [7]:
# 회귀 계수 출력
print('Weight1 =', W1.numpy())
print('Bias1 =', b1.numpy())
print('Weight2 =', W2.numpy())
print('Bias2 =', b2.numpy())

Weight1 = [[-7.127812  -8.760641 ]
 [-7.1037564 -8.603351 ]]
Bias1 = [10.717856   4.0264554]
Weight2 = [[ 15.241033]
 [-14.8807  ]]
Bias2 = [-7.569965]


In [8]:
# predict : test model
# accuracy computation (정확도 측정)
def predict(X):
    return tf.cast(hypothesis(X) > 0.5, dtype = tf.float32)

accuracy = tf.reduce_mean(tf.cast(tf.equal(predict(x_train),y_train),
                                     dtype = tf.float64))
print("Hypothesis:\n",hypothesis(x_train).numpy(), 
      "\nPredict:\n",predict(x_train).numpy(),
      "\nAccuracy:",accuracy.numpy())   # Accuracy: 1.0

end = time.time()
print('elapsed time:',end - start)
# 시간 비교
# @tf.function을 사용하지 않았을 때 : 26.349613904953003
# @tf.function을 모든 함수에  사용했을 때 : 13.266760349273682
# cost_func만 @tf.function 을 사용했을 때 : 13.436646461486816
# 텐서플로는 그래프를 캐시에 저장하기 때문에 함수가 호출될 때 다시 그래프를 생성할 필요가 없게 된다

Hypothesis:
 [[9.5849548e-04]
 [9.9919170e-01]
 [9.9920160e-01]
 [8.0075976e-04]] 
Predict:
 [[0.]
 [1.]
 [1.]
 [0.]] 
Accuracy: 1.0
elapsed time: 26.349613904953003
